# Protein pairs

We use ColabFold MSA for protein pairs pipeline, as it takes shorter time

## Setup

In [ ]:
#| default_exp protein_pairs

In [ ]:
#| export
import os, json, shutil, pandas as pd
from pathlib import Path
from tqdm import tqdm
from itertools import combinations
from pathlib import Path
from af_kit.json import *
from af_kit.docker import *

## Setup

```sh
docker pull sky1ove/alphafold3
```

## Protein pairs

Since protein protein screening involves a lot of proteins, it takes a long time for AF3 default MSA pipeline, so we used colabfold MSA pipeline

In [ ]:
#| export
def get_colabfold_cmd(csv_path,project_name):
    print('Run below in terminal:')
    print(f'\n colabfold_batch {csv_path} msa_{project_name} --msa-only')

In [ ]:
project_name='sdf'

In [ ]:
get_colabfold_cmd('sdf.csv',project_name)

Run below in terminal:

 colabfold_batch sdf.csv msa_sdf --msa-only


## MSA

In [ ]:
#| export
def copy_a3m(a3m_dir: str, # Path to the source directory containing .a3m files.
             dest_dir: str, # Path to the destination directory where files will be copied
             ):
    "Copies all .a3m files from the source directory to the destination directory."
    
    a3m_dir,dest_dir = Path(a3m_dir),Path(dest_dir)
    dest_dir.mkdir(parents=True, exist_ok=True)

    files = list(a3m_dir.glob('*.a3m'))

    for file in tqdm(files, desc="Copying files", unit="file"):
        shutil.copy(file, dest_dir / file.name)

    print(f"Copied {len(files)} a3m files from {a3m_dir} to {dest_dir}")

In [ ]:
copy_a3m(a3m_dir='data',dest_dir='af_input')

Copying files: 100%|██████████| 1/1 [00:00<00:00, 637.53file/s]

Copied 1 a3m files from data to af_input


## Protein-protein input

:::{.callout-important}
Make sure a3m files are under af_input, otherwise it won't detect the files
:::

In [ ]:
#| export
def a3m_to_seq(file_path:Path):
    "Get protein sequence from a3m file"
    return file_path.read_text().splitlines()[2] # protein sequence is located on line 2

In [ ]:
a3m_to_seq(Path(f'af_input/{project_name}/a3m/CD8A.a3m'))

'SQFRVSPLDRTWNLGETVELKCQVLLSNPTSGCSWLFQPRGAAASPTFLLYLSQNKPKAAEGLDTQRFSGKRLGDTFVLTLSDFRRENEGYYFCSALSNSIMYFSHFVPVFLPAKPTTTPAPRPPTPAPTIASQPLSLRPEACRPAAGGAVHTRGLDFACD'

In [ ]:
#| export
def get_protein_subjson(gene_name, a3m_dir=".",idx = 'A',run_template=True):
    "Get subjson (protein part) with colabfold unpairedMSA .a3m path"
    file_path = Path(a3m_dir)/f"{gene_name}.a3m"
    protein_sequence = a3m_to_seq(file_path)
    
    json_data = {
        'id': idx,
        'sequence': protein_sequence, 
        'modifications': [],
        'unpairedMsaPath': str("/root"/file_path), # for docker path, ECD under af_input
        'pairedMsa': '',
        'templates': None if run_template else []
    }

    return json_data

In [ ]:
sub_json = get_protein_subjson('CD8A',a3m_dir=f'af_input/{project_name}/a3m')

In [ ]:
sub_json

{'id': 'A',
 'sequence': 'SQFRVSPLDRTWNLGETVELKCQVLLSNPTSGCSWLFQPRGAAASPTFLLYLSQNKPKAAEGLDTQRFSGKRLGDTFVLTLSDFRRENEGYYFCSALSNSIMYFSHFVPVFLPAKPTTTPAPRPPTPAPTIASQPLSLRPEACRPAAGGAVHTRGLDFACD',
 'modifications': [],
 'unpairedMsaPath': '/root/af_input/sdf/a3m/CD8A.a3m',
 'pairedMsa': '',
 'templates': None}

In [ ]:
#| export
def dump_json_folder(json_data, folder):
    "Save json under a folder"
    file_path = Path(folder)/f"{json_data['name']}.json"
    with open(file_path,'w') as f: json.dump(json_data,f,indent=4)

In [ ]:
#| export
def get_multi_protein_json(gene_list,a3m_dir,run_template=True,save_folder=None):
    'Get json of multiple proteins, with unpaired MSA path indicated (from colabfold MSA)'
    sequences = []
    alphabets = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    for index, gene in enumerate(gene_list):
        sub_json=get_protein_subjson(gene,a3m_dir,idx=alphabets[index],run_template=run_template)
        sequences.append({'protein':sub_json})
    name = '_'.join(gene_list)
    json_data = {
            "name": name,
            "modelSeeds": [1],
            "sequences": sequences,
            "bondedAtomPairs": [],
            "dialect": "alphafold3",
            "version": 2
        }
    if save_folder:
        dump_json_folder(json_data,save_folder)
    return json_data

In [ ]:
AF_input = get_multi_protein_json(['CD8A','CD8A'],
                        a3m_dir=f'af_input/{project_name}/a3m',
                        save_folder=f'af_input/{project_name}')

You can generate a list of json files under a folder.

In [ ]:
AF_input.keys(), len(AF_input['sequences'])

(dict_keys(['name', 'modelSeeds', 'sequences', 'bondedAtomPairs', 'dialect', 'version']),
 2)

In [ ]:
#| export
def generate_pair_df(gene_list,self_pair=True):
    "Unique pair genes in a gene list"
    pairs = list(combinations(gene_list, 2))
    pair_df = pd.DataFrame(pairs,columns=["Gene1", "Gene2"])
    
    if self_pair:
        self_pair_df = pd.DataFrame({'Gene1':gene_list, 'Gene2':gene_list})
        pair_df = pd.concat([pair_df,self_pair_df])

    return pair_df.reset_index(drop=True)

In [ ]:
generate_pair_df(list('ABC'))

,Gene1,Gene2
0,A,B
1,A,C
2,B,C
3,A,A
4,B,B
5,C,C


In [ ]:
df = generate_pair_df(['CD8A'])
df

,Gene1,Gene2
0,CD8A,CD8A


Generate json files first:

In [ ]:
for idx, row in tqdm(df.iterrows(),total=len(df)):
    json_data = get_multi_protein_json([row['Gene1'], row['Gene2']], 
                             a3m_dir=f'af_input/{project_name}/a3m', 
                             save_folder=f'af_input/{project_name}') 

100%|██████████| 1/1 [00:00<00:00, 147.81it/s]


Split them to subfolder:

In [ ]:
split_nfolder(f'af_input/{project_name}')

Distributed 1 files into 4 folders.


## Docker

Todo: Pair proteins

```python
for i in range(4):
    get_docker_command(input_dir=f"af_input/{project_name}/folder_{i}",
                       output_dir=f"af_output/{project_name}",
                       gpus=i)
```

## Read summary confidences json

In [ ]:
#| export
def read_summary_json(json_path):
    "Read json file to dictionary"
    json_path = Path(json_path)
    json_data=json.loads(json_path.read_text())
        
    # Flatten the JSON data into a single row
    row = {"ID": json_path.stem}
    for key, value in json_data.items():
        if isinstance(value, list):
            for idx, sub_value in enumerate(value):
                if isinstance(sub_value, list):
                    for sub_idx, sub_sub_value in enumerate(sub_value):
                        row[f"{key}_{idx}_{sub_idx}"] = sub_sub_value
                else:
                    row[f"{key}_{idx}"] = sub_value
        else:
            row[key] = value
    return row

In [ ]:
data = read_summary_json('data/seq_only_summary_confidences.json')
data

{'ID': 'seq_only_summary_confidences',
 'chain_iptm_0': None,
 'chain_pair_iptm_0_0': 0.72,
 'chain_pair_pae_min_0_0': 0.76,
 'chain_ptm_0': 0.72,
 'fraction_disordered': 0.19,
 'has_clash': 0.0,
 'iptm': None,
 'ptm': 0.72,
 'ranking_score': 0.82}

In [ ]:
#| export
def get_summary_df(output_dir):
    "Pack the summary json from the output folder to the df"
    
    path_list = list(Path(output_dir).rglob('*_summary_confidences.json'))
    print(len(path_list),'summary_confidences.json files detected')
    return pd.DataFrame(list(map(read_summary_json,path_list)))

In [ ]:
out = get_summary_df('data')
out

1 summary_confidences.json files detected


,ID,chain_iptm_0,chain_pair_iptm_0_0,chain_pair_pae_min_0_0,chain_ptm_0,fraction_disordered,has_clash,iptm,ptm,ranking_score
0,seq_only_summary_confidences,None,0.72,0.76,0.72,0.19,0.0,None,0.72,0.82


## Specific for protein pairs

In [ ]:
#| export
def process_summary_df(df,generate_report=False):
    "Post process the json-converted pandas df; remove redundant columns; available for pairs"
    
    df['ID'] = df.ID.str.replace('_summary_confidences','')
    df = df.set_index('ID')

    # drop zero std columns: usually contains chain_pair_pae_min_0_0 and has_clash
    zero_std_cols = df.columns[df.std()<1e-10]
    df=df.drop(columns=zero_std_cols)

    # drop columns with same values
    redundant_columns = []
    
    # Check for columns equal to `iptm`
    for col in ['chain_iptm_0', 'chain_iptm_1', 'chain_pair_iptm_0_1', 'chain_pair_iptm_1_0']:
        if df['iptm'].equals(df[col]):
            redundant_columns.append(col)
    
    # Check for columns equal to `chain_ptm_0`
    if df['chain_ptm_0'].equals(df['chain_pair_iptm_0_0']):
        redundant_columns.append('chain_pair_iptm_0_0')
    
    # Check for columns equal to `chain_ptm_1`
    if df['chain_ptm_1'].equals(df['chain_pair_iptm_1_1']):
        redundant_columns.append('chain_pair_iptm_1_1')
    
    # Drop redundant columns
    df = df.drop(columns=redundant_columns)
    
    if generate_report:
        print('Generating pairplot graph report')
        sns.pairplot(data=df, corner=True)
        plt.savefig("af_report.pdf")
        plt.close()
        print('Export to af_report.pdf')

    # add iptm and ptm
    df['iptm_ptm_add'] = df['iptm']+df['ptm']

    # inter error add
    df['chain_pair_pae_min_add'] = df['chain_pair_pae_min_0_1']+df['chain_pair_pae_min_1_0']

    # rank iptm and ptm and add the rank, this method can ignore the different value range between iptm and ptm
    df['iptm_rnk'],df['ptm_rnk'] = df.iptm.rank(ascending=False), df.ptm.rank(ascending=False)
    df['iptm_ptm_rnk_add'] = df['iptm_rnk']+df['ptm_rnk']

    # combine chain_pair_pae_min & iptm rank
    df['chain_pair_pae_min_add_rnk'] = df['chain_pair_pae_min_add'].rank()
    df['iptm_pae_add_rnk'] = df['chain_pair_pae_min_add_rnk'] + df['iptm_rnk']
    
    return df

In [ ]:
# it's only available for protein pairs
# out2 = process_summary_df(out)

In [ ]:
#| export
def get_top_genes(df,n=30):
    "Get top genes from the metric"
    idxs = set()
    
    big_cols = ['ranking_score', 'iptm', 'iptm_ptm_add']
    small_cols = [
        'iptm_ptm_rnk_add', 
        'chain_pair_pae_min_add', 
        'chain_pair_pae_min_0_1', 
        'chain_pair_pae_min_1_0', 
        'iptm_pae_add_rnk'
    ]
    
    for col in big_cols: idxs.update(df.nlargest(n, col).index)
    for col in small_cols: idxs.update(df.nsmallest(n, col).index)

    return list(idxs)

In [ ]:
# genes = get_top_genes(out2)

In [ ]:
#| export
def get_3d_report(df,index_list, x='iptm',y='ptm',z='chain_pair_pae_min_add',save_dir='af_report'):
    "Plot plotly 3d"
    annotation = df.index.where(df.index.isin(index_list),'').str.split('_').str[1]

    # Create the 3D scatter plot
    fig = go.Figure(data=go.Scatter3d(
        x=df[x],
        y=df[y],
        z=df[z],
        mode='markers+text',
        text=annotation,  # Annotation using the index
        textposition="top center",
        marker=dict(size=8, color='blue', opacity=0.8),
    ))
    
    # Customize layout
    fig.update_layout(
        scene=dict(
            xaxis_title=x,
            yaxis_title=y,
            zaxis_title=z,
        ),
        title='3D Scatter Plot',
        autosize=True,
        height=3000,
    )
    
    fig.write_html(Path(save_dir)/'3d_scatter_plot.html',full_html=True)
    print('Exported the html to 3d_scatter_plot.html')

In [ ]:
# get_3d_report(out2,genes)

In [ ]:
#| export
def get_report(out_dir,save_dir='af_report'):
    
    out = get_summary_df(out_dir)
    out = process_summary_df(out)
    top_genes = get_top_genes(out)
    
    Path(save_dir).mkdir(exist_ok=True)
    get_3d_report(out,top_genes,save_dir=save_dir)
    out.to_csv(Path(save_dir)/'summary_confidences.csv')
    
    return out, top_genes

```python
df_sum, top_genes = get_report('af_output/data','af_report/proteinA')

df_sum.sort_values('iptm_ptm_rnk_add').head(10)
```

In [ ]:
#| export
def copy_file(idx_name, source_dir, dest_dir):
    "Copy all model cif generated by AF3 to the new dest folder"
    source_path = Path(source_dir)/f"{idx_name}/{idx_name}_model.cif"
    dest_path = Path(dest_dir)/f"{idx_name}_model.cif"
    shutil.copy(source_path,dest_path)
    print(f'Copying {str(source_path)} to {str(dest_path)}')

```python
from fastcore.utils import L
copy_file('proA_proB',source_dir='af_output/proA',dest_dir='af_top')
# Or 
L(top_genes).map(copy_file,source_dir='af_output/proA',dest_dir='af_top')
```

## End

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

## Utils

In [ ]:
# #| export
# def split_files_into_subfolders(input_folder: str, nfolder: int = 4):
    
#     "Splits `.a3m` files in a folder into subfolders (folder_0, folder_1, ..., folder_N)."
    
#     input_path = Path(input_folder)
#     if not input_path.is_dir():
#         raise ValueError(f"Input folder {input_folder} does not exist or is not a directory.")

#     # List all `.a3m` files
#     a3m_files = sorted(input_path.glob("*.a3m"))
#     if not a3m_files:
#         print("No `.a3m` files found in the input folder.")
#         return

#     # Create the subfolders
#     subfolders = [input_path / f"folder_{i}" for i in range(nfolder)]
#     for folder in subfolders:
#         folder.mkdir(exist_ok=True)

#     # Distribute the files into the subfolders
#     for idx, file in enumerate(a3m_files):
#         target_folder = subfolders[idx % nfolder]
#         shutil.move(str(file), target_folder / file.name)

#     print(f"Distributed {len(a3m_files)} files into {nfolder} folders.")
